In [10]:
from z3 import *
from itertools import combinations

In [11]:
set_param("parallel.enable", True)

In [12]:
instances_path = "../../instances/"
with open(instances_path + "ins-10.txt") as f:
    lines = [f[:-1] for f in f.readlines()]
    w, n = int(lines[0]), int(lines[1])
    b = [d.split() for d in lines[2:]]
    b = [(int(bw), int(bh)) for (bw, bh) in b]

In [13]:
def at_least_one_np(bool_vars):
    return Or(bool_vars)

def at_most_one_np(bool_vars, name = ""):
    return [Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)]

def exactly_one_np(bool_vars, name = ""):
    return And(at_least_one_np(bool_vars), And(at_most_one_np(bool_vars, name)))

In [14]:
def at_least_one_he(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_he(bool_vars, name):
    if len(bool_vars) <= 4:
        return And(at_most_one_np(bool_vars))
    y = Bool(f"y_{name}")
    return And(And(at_most_one_np(bool_vars[:3] + [y])), And(at_most_one_he(bool_vars[3:] + [Not(y)], name+"_")))

def exactly_one_he(bool_vars, name=""):
    return And(at_most_one_he(bool_vars, name), at_least_one_he(bool_vars))

In [48]:
def vlsi_sat(n, b, w, h, rot=False):
    cells = [[Int(f"cells_{i}_{j}") for j in range(w)] for i in range(h)]
    
    o = Optimize()
    o.add([(cells[i][j] >= 0) for i in range(h) for j in range(w)])
    o.minimize(Sum([cells[i][j] for i in range(h) for j in range(w)]))
    #s = Solver()
    #s.set("sat.pb.solver", "solver")

    # Each block must occur exactly once
    for k in range(1, n + 1):
        (x, y) = b[k - 1]
        cnst = []
        for i in range(h):
            for j in range(w):
                if j + x < w and i + y < h:
                    block = [cells[r][c] == k
                             for r in range(i, i + y) for c in range(j, j + x)]
                    cnst.append(And(block))
                else:
                    cnst.append(False)
        o.add(PbEq([(block_and, 1) for block_and in cnst], 1))
        #o.add(exactly_one([block_and for block_and in cnst]))
    
    # Check if it's SAT
    o.check()
    if o.check() == sat:
        m = o.model()
        #return [(i, j, k) for i in range(h) for j in range(w) 
        #        for k in range(n + 1) if m.evaluate(cells[i][j][k])]
        return o
    return "UNSAT"

In [49]:
h = sum([bh for (bw, bh) in b])

In [ ]:
%%time
exactly_one = exactly_one_np
sol = vlsi_sat(n, b, w, h)

In [38]:
sol

NameError: name 'sol' is not defined